## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [12]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal

import matplotlib.pyplot as plt
%matplotlib inline

import mpld3
mpld3.enable_notebook()


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and
        corresponding reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse
        rate error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))    

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an
    aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return (AggregateErrorMetric(errs, confs), errs, confs)    

def BandpassFilter(signal, bandpass=(40/60,240/60), fs=125):
    """
    Bandpass Filter:
        Filers the input signal, in this case between 40 and 240 BPM

    Args:
        signal:     (np.array) The input signal
        pass_band:  (tuple) The pass band. Frequency components outside the bandpass are filered.
        fs:         (number) The sampling rate of <signal>
        
    Returns:
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(3, bandpass, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)


def Features(signal, fs=125):
    """ 
    Features:
        A featurization of the signal data
    Ags:
        signal:     (np.array) The input signal
        fs:         (number) The sampling rate of <signal>
    returns:
        list of frequencies, fft and weights
    """

    # run the descrete Fourier Transform. it returns a float array
    freqs = np.fft.rfftfreq(len(signal), 1 / fs)

    # Take an FFT of the centered signal
    fft = np.abs(np.fft.rfft(signal - np.mean(signal), len(signal)))
    # filter the range between the lower and higher BPM
    fft[freqs <= (40 / 60)] = 0.0
    fft[freqs >= (240 / 60)] = 0.0

    # the weight of the signal
    weight = np.sum(np.square(signal - np.mean(signal)))

    return freqs, fft, weight

def EstimateMaxPPG(ppg_weight, acc_weight, ppg_frequencies, acc_frequencies, ppg_fft, acc_fft):
    """
    """
    max_ppg = ppg_frequencies[np.argmax(ppg_weight)]
    max_acc = acc_frequencies[np.argmax(acc_weight)]

    lcv = 1       
    while np.abs(max_ppg - max_acc) <= 0 and lcv <=2:
        lcv += 1
        max_ppg = ppg_frequencies[np.argsort(ppg_fft, axis = 0)[-lcv]]
        max_acc = acc_frequencies[np.argsort(acc_fft, axis = 0)[-lcv]]
    
    return max_ppg, max_acc

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Load the reference data, also named as "ground truth" heart rate data
    reference_bpm = sp.io.loadmat(ref_fl)['BPM0']
    
    # filter signals
    ppg  = BandpassFilter(ppg)
    accx = BandpassFilter(accx)
    accy = BandpassFilter(accy)
    accz = BandpassFilter(accz)
    
    # Calculate the acc magnitude
    acc_magnitude = np.sqrt(np.square(accx) + np.square(accy) + np.square(accz))
    
    estimates = []
    confidences = []
    fs = 125
    window_length = 8 * fs
    window_shift = 2 * fs
   
    for i in range(0, len(ppg) - window_length, window_shift):
        
        ppg_freqs, ppg_fft, ppg_weigth = Features(ppg[i:i + window_length], fs)
        acc_freqs, acc_fft, acc_weigth = Features(acc_magnitude[i:i + window_length], fs)

        est_freqs = EstimateMaxPPG(ppg_weigth, acc_weigth, ppg_freqs, acc_freqs, ppg_fft, acc_fft)[0]  
        estimates.append(est_freqs * 60)
        
        # calculate the confidence with a windows frequency of 40/60 (Hz)
        window_f = 40/60
        confidence = np.sum(ppg_fft[(ppg_freqs >= (est_freqs - window_f)) & (ppg_freqs <= (est_freqs + window_f))])/ np.sum(ppg_fft)                
        confidences.append(confidence) 
        
        # Return MEA and confidence.
    errors = np.abs(np.diag(estimates - reference_bpm))
    return np.array(errors), np.array(confidences)



'\n    window_length = 8 * fs\n    window_shift = 2 * fs\n\n    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)\n\n    # Load the reference data, also named as "ground truth" heart rate data\n    reference_bpm = sp.io.loadmat(ref_fl)[\'BPM0\']\n\n    # filter the data using bandpass\n    ppg_bandpass  = BandpassFilter(ppg)\n    accx_bandpass = BandpassFilter(accx)\n    accy_bandpass = BandpassFilter(accy)\n    accz_bandpass = BandpassFilter(accz)\n\n    # Calculate the acc magnitude\n    acc_magnitude = np.sqrt(np.square(accx_bandpass) + np.square(accy_bandpass) + np.square(accz_bandpass))\n    \n    estimate = []\n    confidence = []\n    \n    for i in range(0, len(ppg_bandpass) - window_length, window_shift):\n        ppg_freqs, ppg_fft, ppg_weight = Features(ppg_bandpass[i:i+window_length], fs)\n        acc_freqs, acc_fft, acc_weigth = Features(acc_magnitude[i:i+window_length], fs)\n\n        max_ppg = ppg_freqs[np.argmax(ppg_weight)]\n        max_acc = acc_freqs[np.argmax(acc_

In [13]:
Evaluate()

(19.24238928158741,
 array([71.91079295,  2.39253394,  2.14285714, ...,  2.5       ,
         1.6904    ,  3.4959    ]),
 array([0.5890318 , 0.59509154, 0.5531311 , ..., 0.72588548, 0.70679039,
        0.73644751]))

In [2]:
data_fls, ref_fls = LoadTroikaDataset()
data_fls, ref_fls

(['./datasets/troika/training_data\\DATA_01_TYPE01.mat',
  './datasets/troika/training_data\\DATA_02_TYPE02.mat',
  './datasets/troika/training_data\\DATA_03_TYPE02.mat',
  './datasets/troika/training_data\\DATA_04_TYPE01.mat',
  './datasets/troika/training_data\\DATA_04_TYPE02.mat',
  './datasets/troika/training_data\\DATA_05_TYPE02.mat',
  './datasets/troika/training_data\\DATA_06_TYPE02.mat',
  './datasets/troika/training_data\\DATA_07_TYPE02.mat',
  './datasets/troika/training_data\\DATA_08_TYPE02.mat',
  './datasets/troika/training_data\\DATA_10_TYPE02.mat',
  './datasets/troika/training_data\\DATA_11_TYPE02.mat',
  './datasets/troika/training_data\\DATA_12_TYPE02.mat'],
 ['./datasets/troika/training_data\\REF_01_TYPE01.mat',
  './datasets/troika/training_data\\REF_02_TYPE02.mat',
  './datasets/troika/training_data\\REF_03_TYPE02.mat',
  './datasets/troika/training_data\\REF_04_TYPE01.mat',
  './datasets/troika/training_data\\REF_04_TYPE02.mat',
  './datasets/troika/training_data\

In [3]:
window_length = 8 * fs
window_shift = 2 * fs

ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])


# Load the reference data, also named as "ground truth" heart rate data
reference_bpm = sp.io.loadmat(ref_fls[0])['BPM0']

# filter the data using bandpass
ppg_bandpass  = BandpassFilter(ppg)
accx_bandpass = BandpassFilter(accx)
accy_bandpass = BandpassFilter(accy)
accz_bandpass = BandpassFilter(accz)

# Calculate the acc magnitude
acc_magnitude = np.sqrt(np.square(accx_bandpass) + np.square(accy_bandpass) + np.square(accz_bandpass))

In [4]:
    
estimate = []
confidence = []
    
for i in range(0, len(ppg_bandpass) - window_length, window_shift):
    ppg_freqs, ppg_fft, ppg_weight = Features(ppg_bandpass[i:i+window_length], fs)
    acc_freqs, acc_fft, acc_weight = Features(acc_magnitude[i:i+window_length], fs)
    
    max_ppg = ppg_freqs[np.argmax(ppg_weight)]
    max_acc = acc_freqs[np.argmax(acc_weight)]
            
    j = 1        
    while np.abs(max_ppg - max_acc) <= 0 and j <=2:
        j += 1
        max_ppg = ppg_freqs[np.argsort(ppg_fft, axis = 0)[-j]]
           
    estimate_frequency = max_ppg 
    estimate.append(estimate_frequency * 60)
        
    window_Hz = 40/60
    conf = np.sum(ppg_fft[(ppg_freqs >= (estimate_frequency - window_Hz)) & 
                        (ppg_freqs <= (estimate_frequency + window_Hz))]) / np.sum(ppg_fft)
        
    confidence.append(conf)
        
errors = np.abs(np.diag(estimate - reference_bpm))
    


-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

## Code Description:
A part of the code was provided and shouldn;t be changed. Most of the student code goes into RunPulseRateAlgorithm. Some additional functions constituted as helper code to determine specific features. 

It basically calculates the heart pulse reate from provided PPG and accelorometer signals. The code runs the provided data against ground truth data where it calculates the heart rate and the confidence.

All data is include in a data directory ./datasets/troika/training_data. 

Running the code is done by calling the function Evaluate().

## Data Description
The data from this project comes from the [Cardiac Arrythmia Suppression Trial (CAST)](https://physionet.org/content/crisdb/1.0.0/), which was sponsored by the National Heart, Lung, and Blood Institute (NHLBI). CAST collected 24 hours of heart rate data from ECGs from people who have had a myocardial infarction (MI) within the past two years.<sup>2</sup> This data has been smoothed and resampled to more closely resemble PPG-derived pulse rate data from a wrist wearable.<sup>3</sup>

Two-channel PPG signals, three-axis acceleration signals, and one-channel ECG signals were simultaneously recorded from subjects with age from 18 to 35. For each subject, the PPG signals were recorded from wrist by two pulse oximeters with green LEDs (wavelength: 515nm). Their distance (from center to center) was 2 cm. The acceleration signal was also recorded from wrist by a three-axis accelerometer. Both the pulse oximeter and the accelerometer were embedded in a wristband, which was comfortably worn. The ECG signal was recorded simultaneously from the chest using wet ECG sensors. All signals were sampled at 125 Hz and sent to a nearby computer via Bluetooth.

Each dataset with the similar name 'DATA_01_TYPE01' contains a variable 'sig'. It has 6 rows. The first row is a simultaneous recording of ECG, which is recorded from the chest of each subject. The second row and the third row are two channels of PPG, which are recorded from the wrist of each subject. The last three rows are simultaneous recordings of acceleration data (in x-, y-, and z-axis). 

During data recording, each subject ran on a treadmill with changing speeds. For datasets with names containing 'TYPE01', the running speeds changed as follows:
  * rest(30s) -> 8km/h(1min) -> 15km/h(1min) -> 8km/h(1min) -> 15km/h(1min) -> rest(30s)

For datasets with names containing 'TYPE02', the running speeds changed as follows:
  * rest(30s) -> 6km/h(1min) -> 12km/h(1min) -> 6km/h(1min) -> 12km/h(1min) -> rest(30s)

For each dataset with the similar name 'DATA_01_TYPE01', the ground-truth of heart rate can be calculated from the simultaneously recorded ECG signal (i.e. the first row of the variable 'sig'). For convenience, we also provide the calculated ground-truth heart rate, stored in the datasets with the corresponding name, say 'REF_01_TYPE01'. In each of this kind of datasets, there is a variable 'BPM0', which gives the BPM value in every 8-second time window. Note that two successive time windows overlap by 6 seconds. Thus the first value in 'BPM0' gives the calcualted heart rate ground-truth in the first 8 seconds, while the second value in 'BPM0' gives the calculated heart rate ground-truth from the 3rd second to the 10th second.


## Algorithm Description
1. Load the data from the training data folder 
2. Load the ground truth data from the same folder  
3. The files are differentiated by name. Training data files start with 'data_*' whereas ground truth files start with 'ref_*'
4. Some parameters, such as the windows length and windows shift must be set. Also it important to set the signal rate = 125. Bandpass is the paarameter to filter initial noisy data and is set to 40 to 240 BPM
  * window_length = 8
  * window_shift = 2
  * fs = 125
  * bandpasss = (40/60,240/60)
5. run the BandPassFilter funtion on the loaded signal. It returns filtered data suchs a PPG and ACC magnitude. The ACC_magnitude is the mean of accx, accy, which are the 3 channels of accelorometer data in the x, y, z direction.
6. The function Features, will featurize the bandpass data. The features include, Fourier Transform, Peak values, Frequecies and their specifc weigth. 
7. as last we estimate the pulse rate for the PPG signal and the acc (3 axis). The estimation is then compared with the ground truth. The result calculates the errors and confidence.

## Algorithm performance
The MAE (Mean ABsolute Error) estimates the heart rate in an 8 second window. Adjusting the frequency window parameter and running the algorithm in the Udacity workspace provided a passed result.


















-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 